# Qualitative Difference btw Convergent and Divergent Comments

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import nltk.data
from mod.clustering.TFIDF import TFIDF
import regex as re

sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

df = pd.read_csv("/Volumes/ROY/comp_ling/datasci/SIS/data/reddit_feminism/individual_comparisons-3.csv")
df = df.loc[df['x_user'] != df['y_user']]
df = df.loc[~df['x_user'].isin([np.nan]) & ~df['y_user'].isin([np.nan])]
df['n'] = df['n'].astype(int)
df = df.loc[(df['n'] > 5)]
# df = df.loc[df['y_user'].isin(df['x_user'].unique())]

df['comment_delta'] = (df['x_comment_no'] - df['y_comment_no'])
df = df.loc[
    ((df['comment_delta'] > -21) & (df['comment_delta'] < 21))
    & (df['comment_delta'] != 0)
]
df['comment_delta_abs'] = df['comment_delta'].abs()

# convert user ids to numerical indicator
# user_dic = {x:i for i,x in enumerate(df['x_user'].unique())}
user_dic = {x:i for i,x in enumerate(np.unique(df[['x_user', 'y_user']].values.astype(str)))}
df['x_user'] = df['x_user'].apply(lambda x: user_dic[x])
df['y_user'] = df['y_user'].apply(lambda x: user_dic[x])

# convert subreddit ids to numerical indicator
#   AND check if two comments are from same subreddit
subreddit_dic = {x:i for i,x in enumerate(df['x_subreddit'].unique())}
df['x_subreddit'] = df['x_subreddit'].apply(lambda x: subreddit_dic[x])
df['y_subreddit'] = df['y_subreddit'].apply(lambda x: subreddit_dic[x])
df['same_subreddit'] = (df['x_subreddit'] == df['y_subreddit']).astype(int) + 1

# df['same_post'] = (df['x_sub_id'] == df['y_sub_id']).astype(int) + 1
df['same_post'] = (df['x_post_id'] == df['y_post_id']).astype(int) + 1
df['t_delta'] = (df['xtime'] - df['ytime'])
df['t_delta_abs'] = df['t_delta'].abs()
# df['post_t_delta'] = (df['x_post_time'] - df['y_post_time']).abs()
df['avgH'] = df['H']/df['n']

In [2]:
dft = pd.read_table('/Volumes/ROY/comp_ling/datasci/SIS/data/reddit_feminism/reddit_with_authors.tsv', lineterminator='\n') # import text corpus

In [3]:
cross_posts = False     # just checks across posts in n = {-1,1}
both_crit = True        # checks across posts and time series where n = {(n < -1) (n > 1)}
                        # otherwise, just check in same post, where n = {(n < -1) (n > 1)}

## r/MensLib

In [4]:
if cross_posts:
    dml = df.loc[
        df['x_subreddit'].isin([2])
        & df['y_subreddit'].isin([2])
        & df['comment_delta'].isin([-1,1])
    ]

    dml['y_subreddit'].loc[dml['x_post_id'] == dml['y_post_id']] = 'converged'
    dml['y_subreddit'].loc[dml['x_post_id'] != dml['y_post_id']] = 'did-not-converge'

elif both_crit:
    dml = df.loc[
        df['x_subreddit'].isin([2])
        & df['y_subreddit'].isin([2])
    ]

    dml['y_subreddit'].loc[
        (dml['x_post_id'] == dml['y_post_id']) 
        & df['comment_delta'].isin([-1,1])
    ] = 'converged'
    
    dml['y_subreddit'].loc[~dml['y_subreddit'].isin(['converged'])] = 'did-not-converge'

else:
    dml = df.loc[
        df['x_subreddit'].isin([2])
        & df['y_subreddit'].isin([2])
        & df['same_post'].isin([2])
    ]
    dml['y_subreddit'].loc[dml['comment_delta'].isin([-1,1])] = 'converged'
    dml['y_subreddit'].loc[~dml['comment_delta'].isin([-1,1])] = 'did-not-converge'


/var/folders/bb/jfrx00ms7pv_jyvncsrvvlvm0000gn/T/ipykernel_5452/1512609551.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dml['y_subreddit'].loc[
/var/folders/bb/jfrx00ms7pv_jyvncsrvvlvm0000gn/T/ipykernel_5452/1512609551.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dml['y_subreddit'].loc[~dml['y_subreddit'].isin(['converged'])] = 'did-not-converge'


set up the data for TF-IDF

In [5]:
dml_ = []
for xid in dml['x'].unique():
    text = dft['body'].loc[int(xid)]
    
    sel = dml['x'].isin([xid])
    
    minima = dml['avgH'].loc[sel].values.argmin()
    label = dml['y_subreddit'].loc[sel].values[minima]
    dml_ +=[[xid, str(text), label]]
dml_ = pd.DataFrame(np.array(dml_), columns=['x', 'body', 'label'])

In [6]:
ml_tfidf = TFIDF(dml_, 'body', 'label')

In [7]:
ml_tfidf.extract_top_n_words_per_topic(5)

{'converged': [('movies', 0.03336759803473893),
  ('feminine', 0.027945022814729582),
  ('masculine', 0.01993597490440445),
  ('10', 0.01969136354458742),
  ('like', 0.015837781319078485)],
 'did-not-converge': [('people', 0.010915101842373758),
  ('think', 0.010784399421608366),
  ('just', 0.010689894455390753),
  ('quote', 0.010649027606887535),
  ('like', 0.010623469675584508)]}

In [8]:
df_ml_tf_long, df_ml_tf_short = [],[]
for k,v in ml_tfidf.extract_top_n_words_per_topic(5).items():
    sub = dml_.loc[dml_['label'].isin([k])]
    for w in v:
        sel = sub['body'].apply(lambda x: w[0] in x)
        texts = sub['body'].loc[sel].values

        body = [[sent for sent in sent_tokenizer.tokenize(text) if w[0] in sent][0] for text in texts]

        if len(body) > 3:
            body = np.random.choice(body, size=(3,), replace=False)

        df_ml_tf_short += [[k, w[0], sent.replace('\n', ' ').replace('<QUOTE>', '')] for sent in body]
        df_ml_tf_long += [[k, w[0], text.replace('\n', ' ').replace('<QUOTE>', '')] for text in texts]

df_ml_tf_long, df_ml_tf_short = np.array(df_ml_tf_long), np.array(df_ml_tf_short)
df_ml_tf_long, df_ml_tf_short = pd.DataFrame(df_ml_tf_long, columns=['label', 'lexeme', 'example']), pd.DataFrame(df_ml_tf_short, columns=['label', 'lexeme', 'example'])

In [9]:
df_ml_tf_short

,label,lexeme,example
0,converged,movies,being a good parent could be seen as tradition...
1,converged,movies,what types of movies do you watch?
2,converged,movies,if you perspective is based entirely on the mo...
3,converged,feminine,being strong and powerful may be traits associ...
4,converged,feminine,there's not enough representation of feminine ...
5,converged,feminine,i think a lot of it stems from the way our soc...
6,converged,masculine,i actually would argue that we don't have masc...
7,converged,masculine,you have very strict idea of what masculine an...
8,converged,masculine,by not being traditionally masculine.
9,converged,10,🥺 these are all when i was 9-10 years old.


In [10]:
convert_to_string = '\hline \\\ label & term & examples \\\ \hline\hline '
for label in df_ml_tf_short['label'].unique():
    l_sel = df_ml_tf_short['label'].isin([label])
    
    for word in df_ml_tf_short['lexeme'].loc[l_sel].unique():
        w_sel = df_ml_tf_short['lexeme'].isin([word])

        convert_to_string += ' ' + label + ' & '
        convert_to_string += word + ' & '
        convert_to_string += '\\begin{itemize} \item ' + ' \item '.join(df_ml_tf_short['example'].loc[l_sel & w_sel].apply(lambda x: x.replace('$', '\$').replace('%', '\%').replace('\n', ' ')).values) + '\end{itemize} \\\ \hline '

file = open('/Volumes/ROY/comp_ling/datasci/SIS/ml-table.txt', 'w')
file.write(convert_to_string)
file.close()

## r/MensRights

In [11]:
if cross_posts:
    dmr = df.loc[
        df['x_subreddit'].isin([1])
        & df['y_subreddit'].isin([1])
        & df['comment_delta'].isin([-1,1])
    ]

    dmr['y_subreddit'].loc[dmr['x_post_id'] == dmr['y_post_id']] = 'converged'
    dmr['y_subreddit'].loc[dmr['x_post_id'] != dmr['y_post_id']] = 'did-not-converge'

elif both_crit:
    dmr = df.loc[
        df['x_subreddit'].isin([1])
        & df['y_subreddit'].isin([1])
    ]

    dmr['y_subreddit'].loc[
        (dmr['x_post_id'] == dmr['y_post_id']) 
        & df['comment_delta'].isin([-1,1])
    ] = 'converged'
    
    dmr['y_subreddit'].loc[~dmr['y_subreddit'].isin(['converged'])] = 'did-not-converge'

else:
    dmr = df.loc[
        df['x_subreddit'].isin([1])
        & df['y_subreddit'].isin([1])
        & df['same_post'].isin([2])
    ]
    dmr['y_subreddit'].loc[dmr['comment_delta'].isin([-1,1])] = 'converged'
    dmr['y_subreddit'].loc[~dmr['comment_delta'].isin([-1,1])] = 'did-not-converge'

/var/folders/bb/jfrx00ms7pv_jyvncsrvvlvm0000gn/T/ipykernel_5452/1653310678.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmr['y_subreddit'].loc[
/var/folders/bb/jfrx00ms7pv_jyvncsrvvlvm0000gn/T/ipykernel_5452/1653310678.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dmr['y_subreddit'].loc[~dmr['y_subreddit'].isin(['converged'])] = 'did-not-converge'


set up the data for TF-IDF

In [12]:
dmr_ = []
for xid in dmr['x'].unique():
    text = dft['body'].loc[int(xid)]
    
    sel = dmr['x'].isin([xid])
    
    minima = dmr['avgH'].loc[sel].values.argmin()
    label = dmr['y_subreddit'].loc[sel].values[minima]
    dmr_ +=[[xid, text, label]]
dmr_ = pd.DataFrame(np.array(dmr_), columns=['x', 'body', 'label'])

In [13]:
mr_tfidf = TFIDF(dmr_, 'body', 'label')

In [14]:
mr_tfidf.extract_top_n_words_per_topic(5)

{'converged': [('good', 0.043084396818067215),
  ('require', 0.02879498050999408),
  ('people', 0.028373382080375573),
  ('single', 0.02496863322784036),
  ('boundaries', 0.023476640128053495)],
 'did-not-converge': [('men', 0.015613955772532257),
  ('just', 0.015132880805332934),
  ('like', 0.015099073591934938),
  ('don', 0.014977556605801193),
  ('women', 0.014336620017402912)]}

In [14]:
df_mr_tf_long, df_mr_tf_short = [],[]
for k,v in mr_tfidf.extract_top_n_words_per_topic(5).items():
    sub = dmr_.loc[dmr_['label'].isin([k])]
    for w in v:
        sel = sub['body'].apply(lambda x: w[0] in x)
        texts = sub['body'].loc[sel].values

        body = [[sent for sent in sent_tokenizer.tokenize(text) if w[0] in sent][0] for text in texts]

        if len(body) > 3:
            body = np.random.choice(body, size=(3,), replace=False)

        df_mr_tf_short += [[k, w[0], sent.replace('\n', ' ').replace('<QUOTE>', '')] for sent in body]
        df_mr_tf_long += [[k, w[0], text.replace('\n', ' ').replace('<QUOTE>', '')] for text in texts]

df_mr_tf_long, df_mr_tf_short = np.array(df_mr_tf_long), np.array(df_mr_tf_short)
df_mr_tf_long, df_mr_tf_short = pd.DataFrame(df_mr_tf_long, columns=['label', 'lexeme', 'example']), pd.DataFrame(df_mr_tf_short, columns=['label', 'lexeme', 'example'])

In [15]:
convert_to_string = '\hline \\\ label & term & examples \\\ \hline\hline '
for label in df_mr_tf_short['label'].unique():
    l_sel = df_mr_tf_short['label'].isin([label])
    
    for word in df_mr_tf_short['lexeme'].loc[l_sel].unique():
        w_sel = df_mr_tf_short['lexeme'].isin([word])

        convert_to_string += ' ' + label + ' & '
        convert_to_string += word + ' & '
        convert_to_string += '\\begin{itemize} \item ' + ' \item '.join(df_mr_tf_short['example'].loc[l_sel & w_sel].apply(lambda x: x.replace('$', '\$').replace('%', '\%').replace('\n', ' ')).values) + '\end{itemize} \\\ \hline '

file = open('/Volumes/ROY/comp_ling/datasci/SIS/mr-table.txt', 'w')
file.write(convert_to_string)
file.close()